In [ ]:
# download the dataset

import os

import tensorflow as tf

dataset_path = os.path.join(
    os.getcwd(),
    'dataset'
)

_ = tf.keras.utils.get_file(
    origin='https://github.com/fabiocarrara/deep-parking/releases/download/archive/CNR-EXT-Patches-150x150.zip', #'http://cnrpark.it/dataset/CNR-EXT-Patches-150x150.zip',
    cache_subdir=os.path.join(dataset_path, 'CNR-EXT-Patches-150x150'),
    extract=True,
)

In [ ]:
# create variables for dataset paths

cnr_ext_path = os.path.join(dataset_path, 'CNR-EXT-Patches-150x150')

labels_path = os.path.join(cnr_ext_path, 'LABELS')
patches_path = os.path.join(cnr_ext_path, 'PATCHES')

train_txt_path = os.path.join(labels_path, 'train.txt')
val_txt_path = os.path.join(labels_path, 'val.txt')
test_txt_path = os.path.join(labels_path, 'test.txt')

In [ ]:
# create function to load label data from the label .txt files

import pandas as pd

def load_labels(txt_path, split):
  df = pd.read_csv(
      txt_path,
      sep=' ',
      names=['path', 'label']
  )

  df_absolute_paths = df['path'].apply(lambda path: os.path.join(patches_path, path))

  df = df.assign(path=df_absolute_paths)
  df = df.assign(split=split)

  return df

In [ ]:
# load the label files and concatenate into a single dataframe

train_df = load_labels(train_txt_path, 'train')
val_df = load_labels(val_txt_path, 'val')
test_df = load_labels(test_txt_path, 'test')

df = pd.concat([train_df, val_df, test_df])

df.head()

In [ ]:
# copy the images to the

import shutil

def move_sample(row):
  dest_folder = os.path.join(dataset_path, row['split'], str(row['label']))
  os.makedirs(dest_folder, exist_ok=True)

  shutil.copy(row['path'], dest_folder)

_ = df.apply(move_sample, axis='columns')

In [ ]:
def load_dataset(split):
  return tf.keras.utils.image_dataset_from_directory(
    os.path.join(dataset_path, split),
    color_mode='grayscale',
    label_mode='categorical',
    batch_size=32,
    image_size=(120, 120),
    shuffle=True,
    seed=42
)

train_ds = load_dataset('train')
val_ds = load_dataset('val')
test_ds = load_dataset('test')

In [ ]:
ALPHA = 0.25
DROPOUT = 0.10

mobilenet_025_96 = tf.keras.applications.mobilenet.MobileNet(
    input_shape=(96, 96, 1),
    alpha=ALPHA,
    dropout=DROPOUT,
    weights=None,
    pooling='avg',
    classes=2,
)

mobilenet_025_96.summary()

In [ ]:
def modify_mobilenet_for_rp2040(mobilenet_model):
  input_type_spec = mobilenet_model.layers[0].input.type_spec

  input = tf.keras.Input(shape=(input_type_spec.shape[1:]))
  output = input

  for layer in mobilenet_model.layers[1:]:
    if (isinstance(layer, tf.keras.layers.Reshape)):
      print("replacing Reshape", layer.name)
      output = tf.keras.layers.Flatten()(output)
    else:
      output = layer(output)

  return tf.keras.Model(input, output)


mobilenet_025_96 = modify_mobilenet_for_rp2040(mobilenet_025_96)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal", seed=42),
  tf.keras.layers.RandomRotation(0.2, seed=42),
  tf.keras.layers.RandomZoom(0.2, seed=42),
  tf.keras.layers.RandomContrast(0.3, seed=42),
  tf.keras.layers.RandomBrightness(0.3, seed=42),
  tf.keras.layers.GaussianNoise(0.01, seed=42),
])

In [ ]:
model = tf.keras.Sequential([
  data_augmentation,
  tf.keras.layers.Resizing(96, 96, crop_to_aspect_ratio=True),
  mobilenet_025_96
])

In [ ]:
callbacks = [
  tf.keras.callbacks.EarlyStopping(
      monitor='val_loss',
      patience=3,
      verbose=0,
      restore_best_weights=True,
  )
]

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
train_ds_cached_shuffled = train_ds.cache().shuffle(1024, seed=42).prefetch(tf.data.AUTOTUNE)
val_ds_cached = val_ds.cache().prefetch(tf.data.AUTOTUNE)

history = model.fit(
    train_ds_cached_shuffled,
    validation_data=val_ds_cached,
    epochs=10,
    batch_size=32,
    callbacks=callbacks
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
model.save('model')

In [ ]:
mobilenet_025_96.save('mobilenet_025_96_model')

In [ ]:
import numpy as np

def image_resize_96_96(x, y):
  x = tf.image.resize(x, (96, 96))
  x = tf.expand_dims(x, axis=0)

  return x, y

val_ds_96_96 = val_ds.unbatch().map(image_resize_96_96)

def representative_dataset():
  for image, label in val_ds_96_96.take(1000):
    yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model(
  tf.keras.Model(
        inputs=[
            mobilenet_025_96.layers[0].input
        ],
        outputs=tf.concat([
            mobilenet_025_96.outputs[0],
            mobilenet_025_96.layers[-2].output,
        ], axis=1)
    )
)
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [ tf.lite.OpsSet.TFLITE_BUILTINS_INT8 ]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.float32

tflite_quant_model = converter.convert()

In [ ]:
with open('mobilenet_025_96.tflite', 'wb') as output:
  print(len(tflite_quant_model))
  output.write(tflite_quant_model);


In [ ]:
interpreter = tf.lite.Interpreter('mobilenet_025_96.tflite')
interpreter.allocate_tensors()

print(interpreter.get_input_details())
print(interpreter.get_output_details())

for x, y in train_ds.unbatch().take(10):
  x = tf.image.resize(x, (96, 96))
  x = tf.expand_dims(x, axis=0)
  x -= 128
  x = tf.cast(x, tf.int8)

  print(x.shape, x.dtype, y)

  interpreter.set_tensor(0, x)
  interpreter.invoke()

  print(
      interpreter.get_tensor(
          interpreter.get_output_details()[0]['index']
      )
  )


In [ ]:
!apt-get install -y xxd

In [ ]:
%%shell
echo "alignas(8) const unsigned char tflite_model[] = {" > tflite_model.h
cat mobilenet_025_96.tflite | xxd -i                                >> tflite_model.h
echo "};"                                                >> tflite_model.h